In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import math
path = Path('/home/jwmun/project_deeplearning')
train_path = path.joinpath('train.csv')
test_path = path.joinpath('test.csv')
submission_path = path.joinpath('submission_popular.csv')
metadata_path = path.joinpath('item_metadata.csv')

In [2]:
print(train_path)
print(test_path)

/home/jwmun/project_deeplearning/train.csv
/home/jwmun/project_deeplearning/test.csv


In [3]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_meta = pd.read_csv(metadata_path)
df_submission = pd.read_csv(metadata_path)

In [166]:
action_list = set(df_train['action_type'].values)
reference_list = set(df_train['reference'].values)
platform_list = set(df_train['platform'].values)
city_list = set(df_train['city'].values)
device_list = set(df_train['device'].values)
filter_list = set(df_train['current_filters'].values)

In [42]:
'''
train_data_split = []
temp_data = []
for i in range(train_size):
    if(i==0):
        temp_data.append(train_data[i])
    if(i>0):
        if(train_data[i,3] > train_data[i-1,3]):
            temp_data.append(train_data[i])
        else:
            train_data_split.append(np.array(temp_data))
            temp_data = []
            temp_data.append(train_data[i])
'''

In [70]:
'''
train_label = []
true_data_num = 0
for i in range(len(train_data_split)):
    #datalength = len(trin_data_split[i])
    if(train_data_split[i][-1,4] == 'clickout item'):
        true_data_num +=1
print(true_data_num)
'''

655250


In [72]:
'''
train_label = []
true_data_num = 0
for i in range(len(train_data_split)):
    datalength = len(train_data_split[i])
    for j in range(datalength):
        if(train_data_split[i][j,4] == 'clickout item'):
            true_data_num +=1
            break

print(true_data_num)
'''

826874


In [105]:
'''
train_label = []
true_data_num = 0
train_data_preprocess = []
for i in range(len(train_data_split)):
    final_click = 0
    have_click = False
    datalength = len(train_data_split[i])
    for j in range(datalength):
        if(train_data_split[i][j,4] == 'clickout item'):
            final_click = j
            have_click = True
    if(final_click < datalength-1 and have_click):
        train_data_preprocess.append(np.delete(train_data_split[i],np.s_[final_click+1:],0))
    elif(have_click):
        train_data_preprocess.append(train_data_split[i])
'''

In [6]:
#테스트 데이터에서 마지막 reference가 nan인지 확인
def check_data(data):
    max_error = 0
    datalength = len(data)
    for i in range(datalength):
        if(isinstance(data[i][-1,5], float)):
            if(math.isnan(data[i][-1,5]) == True):
                pass
            else:
                print('error')
                print(data[i])
                max_error +=1
        else:
            print('error')
            print(data[i])
            max_error +=1
        if(max_error > 10):
            break

In [7]:
#테스트 데이터에서 reference가 nan일때 그 전 action이 click out인지 확인
def check_data_alpha(data):
    max_error = 0
    datalength = len(data)
    for i in range(datalength):
        for j in range(len(data[i])):
            if(isinstance(data[i][j,5], float)):
                if(math.isnan(data[i][j,5]) == True):
                     if(data[i][j,4] != 'clickout item'):
                        print(data[i])
                        max_error +=1
        
        if(max_error > 10):
            break

In [8]:
#테스트 데이터에서 reference에 하나라도 nan이 있는지 판별 있다면 그 중 적어도 하나는 action이 click out인지 판별
def check_data_beta(data):
    max_error = 0
    datalength = len(data)
    for i in range(datalength):
        error_data_number = 0
        click_check = False
        for j in range(len(data[i])):
            if(isinstance(data[i][j,5], float) and math.isnan(data[i][j,5]) == True):
                error_data_number += 1
        if(error_data_number == 0):
            print(data[i])
            print('error')
            max_error +=1
        if(error_data_number != 0):
            for j in range(len(data[i])):
                if(isinstance(data[i][j,5], float) and math.isnan(data[i][j,5]) == True):
                    if(data[i][j,4] == 'clickout item'):
                        click_check = True
        if(click_check == False):
            print('error')
            print(data[i])
            max_error +=1
            #print(data[i])
            #print('datas')
        if(max_error > 10):
            break

In [10]:
# 트레이닝 데이터를 세션 단위로 나눔
def make_data(data): 
    data_split = []
    temp_data = []
    train_size = len(data)
    action_index = 1
    for i in range(train_size):
        if(i==0):
            temp_data.append(data[i])
        if(i>0):
            if(data[i,action_index] == data[i-1,action_index]):
                temp_data.append(data[i])
            else:
                data_split.append(np.array(temp_data))
                temp_data = []
                temp_data.append(data[i])
    return data_split

In [135]:
train_data_split = make_data(df_train.values)

In [14]:
#트레이닝 데이터에서 click out이 존재하는 데이터만 뽑아냄
#트레이닝 데이터에서 끝부분이 click out이 되도록 뒷부분을 자름
def data_preprocess(data): 
    new_data = []
    for i in range(len(data)):
        check_data = False
        final_click = 0
        for j in range(len(data[i])):
            if(data[i][j,4] =='clickout item'):
                final_click = j
                check_data = True
        if(final_click < len(data[i])-1 and check_data):
            new_data.append(np.delete(data[i],np.s_[final_click+1:],0))
        elif(check_data):
            new_data.append(data[i])
    return new_data

In [136]:
train_data_preprocess = data_preprocess(train_data_split)

In [138]:
len(train_data_preprocess) #총 데이터 개수

826874

In [140]:
train_data_preprocess[:10]

[array([['00RL8Z82B2Z1', 'aff3928535f48', 1541037460, 1, 'search for poi',
         'Newtown', 'AU', 'Sydney, Australia', 'mobile', nan, nan, nan],
        ['00RL8Z82B2Z1', 'aff3928535f48', 1541037522, 2,
         'interaction item image', '666856', 'AU', 'Sydney, Australia',
         'mobile', nan, nan, nan],
        ['00RL8Z82B2Z1', 'aff3928535f48', 1541037522, 3,
         'interaction item image', '666856', 'AU', 'Sydney, Australia',
         'mobile', nan, nan, nan],
        ['00RL8Z82B2Z1', 'aff3928535f48', 1541037532, 4,
         'interaction item image', '666856', 'AU', 'Sydney, Australia',
         'mobile', nan, nan, nan],
        ['00RL8Z82B2Z1', 'aff3928535f48', 1541037532, 5,
         'interaction item image', '109038', 'AU', 'Sydney, Australia',
         'mobile', nan, nan, nan],
        ['00RL8Z82B2Z1', 'aff3928535f48', 1541037532, 6,
         'interaction item image', '666856', 'AU', 'Sydney, Australia',
         'mobile', nan, nan, nan],
        ['00RL8Z82B2Z1', 'aff392

In [164]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [165]:
items = df_meta['item_id'].values
properties = df_meta['properties'].values
all_items_test = []
for i in range(len(properties)):
    p_data = properties[i].split('|')
    all_items_test += p_data
set_items = set(all_items_test)
list_items = list(set_items)
list_items.sort()
def embedding_item_diction():
    all_items = {}
    for i in range(len(properties)):
        all_items[items[i]] = properties[i].split('|')
    embedding_items = {}
    for keys in all_items.keys():
        item_embeddings = [0]*len(list_items)
        for i in range(len(list_items)):
            if list_items[i] in all_items[keys]:
                item_embeddings[i] = 1
        embedding_items[keys] = item_embeddings
    return embedding_items
reference_diction = embedding_item_diction()

In [167]:
test = list(filter_list)
all_filter_list = []
for i in range(len(test)):
    if i>0:
        all_filter_list += test[i].split('|')
all_filter_list = set(all_filter_list)

In [168]:
def filter_embedding(data, version):
    embedding_data = [0]*202
    if version == 1:
        datalist = data.split('|')
        for filters in datalist:
            embedding_data[list(all_filter_list).index(filters)] = 1
        return embedding_data

In [169]:
def one_hot_encoding(number):
    return [int(i==number) for i in range(10)]

In [170]:
def embedding(data, action_index):
    if action_index == 0: #user_id dim 0
        return []
    
    if action_index == 1: # session_id dim 0
        return []
    
    if action_index == 2: # timestep dim 0
        return []
    
    if action_index == 3: #step dim 1
        return [int(data)]
    
    if action_index == 4: #action_type dim 10
        return one_hot_encoding(list(action_list).index(data))
    
    if action_index == 5: #reference dim 157
        if RepresentsInt(data):
            return reference_diction[int(data)]
        else:
            return [0]*157
        
    if action_index == 6: #platform dim 55
        return one_hot_encoding(list(platform_list).index(data))
    
    if action_index == 7: #city dim 0
        return []
    
    if action_index == 8: #device dim 3
        return one_hot_encoding(list(device_list).index(data))
    
    if action_index == 9: #current_filters dim 202
        if(isinstance(data, float) and math.isnan(data) == True):
            return [0]*202
        else: 
            return filter_embedding(data, 1)
        
    if action_index == 10: #impressions dim 3925
        if(isinstance(data, float) and math.isnan(data) == True):
            return [0]*3925
        else:
            impression_embedding = []
            impression_list = data.split('|')
            for impression in impression_list:
                impression_embedding += reference_diction[int(impression)]
            return impression_embedding
        
    if action_index == 11: #prices dim 25
        if(isinstance(data, float) and math.isnan(data) == True):
            return [0]*25
        else:
            price_embedding = []
            for prices in data.split('|'):
                price_embedding.append(int(prices))
            return price_embedding

In [171]:
def check_last_click(data):
    data_number = 0
    for i in range(len(data)):
        if(data[i][-1,4] == 'clickout item'):
            data_number +=1
    return data_number

In [172]:
action_type_length = 10
reference_length = 157
platform_length = 55
device_length = 3
current_filters_length = 202
impresssions_length = 3925
prices_length = 25
def addVector(vector1, vector2, coef, action_num):
    new_action_type = []
    new_reference = []
    new_platform = []
    new_device = []
    new_filters = []
    new_impressions = []
    new_prices = []
    print(len(vector1))
    print(len(vector2))   
    for i in range(10):
        new_action_type.append(vector1[i] + np.power(coef,action_num)*vector2[i])
    
    for i in range(10, 167):
        new_action_type.append(vector1[i] + np.power(coef,action_num)*vector2[i])
        
    for i in range(167, 222):
        new_action_type.append(vector1[i])
    
    for i in range(222, 225):
        new_action_type.append(vector1[i])
        
    for i in range(225, 427):
        new_action_type.append(vector1[i] + np.power(coef,action_num)*vector2[i])
    
    for i in range(427, 4352):
        new_action_type.append(vector1[i] + np.power(coef,action_num)*vector2[i])
        
    for i in range(4352, 4377):
        new_action_type.append(vector1[i] + np.power(coef,action_num)*vector2[i])
        
    return new_action_type

In [173]:
def MakeEmbeddingVector(data):
    #embedding_vector = []
    #temp_vector = []
    session_vector = [0]*4377 # session data embedding(length 10+157+55+3+202+3925+25=4377)
    item_vector = [] # item data embedding
    num_action = len(data)
    time_delay_coef = 0.1
    for i in range(num_action-1, -1, -1): #거꾸로
        timestep_vector = []
        for j in range(12):
            timestep_vector += embedding(data[i,j], j) 
        if(i == num_action-1):
            for idx in range(10, 167):
                timestep_vector[idx] = 0 # reference 부분을 0으로 만듦
            item_vector = embedding(data[i, 5], 5) #item vector 생성
        session_vector = addVector(session_vector, timestep_vector, time_delay_coef, num_action-1-i) #session vector 생성   
            
    return session_vector, item_vector

In [174]:
sevec, itemvec = MakeEmbeddingVector(train_data_preprocess[0])

4377
4340


IndexError: list index out of range

In [131]:
for i in range(10,-1, -1):
    print(i)

10
9
8
7
6
5
4
3
2
1
0


In [ ]:
def mapping(data, action_index):
    if action_type == 'action_type':
        return one_hot_encoding(action_list.index(data))
    if action_type == 'reference':
        return 

In [28]:
def one_hot_encoding(number):
    return [int(i==number) for i in range(10)]

In [52]:
all_items_test = []
for i in range(len(properties)):
    p_data = properties[i].split('|')
    all_items_test += p_data
set_items = set(all_items_test)

In [110]:
all_items = {}
properties = df_meta['properties'].values
items = df_meta['item_id'].values
for i in range(len(properties)):
    all_items[items[i]] = properties[i].split('|')

In [115]:
embedding_items = {}
list_items = list(set_items)
list_items.sort()
for keys in all_items.keys():
    item_embeddings = [0]*len(list_items)
    for i in range(len(list_items)):
        if list_items[i] in all_items[keys]:
            item_embeddings[i] = 1
    embedding_items[keys] = item_embeddings

In [45]:
items = df_meta['item_id'].values
properties = df_meta['properties'].values
all_items_test = []
for i in range(len(properties)):
    p_data = properties[i].split('|')
    all_items_test += p_data
set_items = set(all_items_test)
list_items = list(set_items)
list_items.sort()
def embedding_item_diction():
    all_items = {}
    for i in range(len(properties)):
        all_items[items[i]] = properties[i].split('|')
    embedding_items = {}
    for keys in all_items.keys():
        item_embeddings = [0]*len(list_items)
        for i in range(len(list_items)):
            if list_items[i] in all_items[keys]:
                item_embeddings[i] = 1
        embedding_items[keys] = item_embeddings
    return embedding_items
reference_diction = embedding_item_diction()

In [46]:
reference_diction = embedding_item_diction()

In [48]:
reference_diction[5101]

[0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1]

In [50]:
len(reference_diction[5101])

157

In [54]:
len(set_items)

157

In [114]:
list_items

['1 Star',
 '2 Star',
 '3 Star',
 '4 Star',
 '5 Star',
 'Accessible Hotel',
 'Accessible Parking',
 'Adults Only',
 'Air Conditioning',
 'Airport Hotel',
 'Airport Shuttle',
 'All Inclusive (Upon Inquiry)',
 'Balcony',
 'Bathtub',
 'Beach',
 'Beach Bar',
 'Beauty Salon',
 'Bed & Breakfast',
 'Bike Rental',
 'Boat Rental',
 'Body Treatments',
 'Boutique Hotel',
 'Bowling',
 'Bungalows',
 'Business Centre',
 'Business Hotel',
 'Cable TV',
 'Camping Site',
 'Car Park',
 'Casa Rural (ES)',
 'Casino (Hotel)',
 'Central Heating',
 'Childcare',
 'Club Hotel',
 'Computer with Internet',
 'Concierge',
 'Conference Rooms',
 'Convenience Store',
 'Convention Hotel',
 'Cosmetic Mirror',
 'Cot',
 'Country Hotel',
 'Deck Chairs',
 'Design Hotel',
 'Desk',
 'Direct beach access',
 'Diving',
 'Doctor On-Site',
 'Eco-Friendly hotel',
 'Electric Kettle',
 'Excellent Rating',
 'Express Check-In / Check-Out',
 'Family Friendly',
 'Fan',
 'Farmstay',
 'Fitness',
 'Flatscreen TV',
 'Free WiFi (Combined)',
 

In [111]:
len(list_items)

157

In [124]:
type(embedding_items)

dict

In [118]:
embedding_items[5101]

[0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1]